# Animations for the Dumped Oscillator

- set up and solve differential equations
- solve for constants given initial conditions
- create animation

In [1]:
from sympy import *
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys

init_printing(use_latex=True)

In [2]:
def solve_do(omega_0, xi, init_pos, init_vel):
    t, w = symbols("t \\omega_0", positive=True)
    d = symbols("\\xi", real=True)
    x = Function("x", real=True)
    eq = x(t).diff(t,t) + 2*d*w*x(t).diff(t) + w**2*x(t)
    eq = eq.subs(w, omega_0).subs(d, xi)
    sol = dsolve(eq,x(t))

    C1, C2 = symbols("C1, C2")
    const = solve([sol.rhs.subs(t,0) - init_pos, sol.rhs.diff(t).subs(t, 0)-init_vel], [C1,C2])
    initvalue_solution = sol.rhs.subs(C1, const[C1]).subs(C2, const[C2])
    
    F = lambdify(t, initvalue_solution, "numpy")
    
    return F
    

In [3]:
def generate_animation(fname, params1, params2, t_max=400, x_range=(-1.2,1.1)):
    global points1, points2,  c
    
    omega1    = params1["omega0"]
    xi1       = params1["xi"]
    init_pos1 = params1["init_pos"]
    init_vel1 = params1["init_vel"]
    
    omega2    = params2["omega0"]
    xi2       = params2["xi"]
    init_pos2 = params2["init_pos"]
    init_vel2 = params2["init_vel"]

    F1 = solve_do(omega1, xi1, init_pos1, init_vel1)
    F2 = solve_do(omega2, xi2, init_pos2, init_vel2)
    
    fig = plt.figure(figsize=(7,3))
    ax = fig.add_subplot(111)
    s1_main   = plt.scatter(F1(0),0 , color="red", alpha=0.5, s=100)
    s1_points = plt.scatter([], [] , color="blue", alpha=0.5, s=20)
    s2_main   = plt.scatter(F2(0),0 , color="red", alpha=0.5, s=100)
    s2_points = plt.scatter([], [] , color="blue", alpha=0.5, s=20)
    text     = ax.text (x_range[0]+(x_range[1]-x_range[0])/2,.55, "$t=0 secstou", size=10)#str(sats[i]), size=8)

    text1     = ax.text (x_range[0]+.2,-.35, "$\omega_0=%.2f"%omega1+", \\xi=%.3f"%xi1+
                         ", x_0=%.2f"%init_pos1+", v_0=%.2f$"%init_vel1, size=10)#str(sats[i]), size=8)
    text2     = ax.text (x_range[0]+.2,.35, "$\omega_0=%.2f"%omega2+", \\xi=%.3f"%xi2+
                         ", x_0=%.2f"%init_pos2+", v_0=%.2f$"%init_vel2, size=10)#str(sats[i]), size=8)
    
    
    plt.xlim(x_range[0],x_range[1])
    plt.ylim(-0.5,0.5)

    points1 = []
    points2 = []

    c=1
    print "frame number:",
    def update(time):
        global points1, points2, c;  
        if c%10==0: print c, 
        sys.stdout.flush(); c+=1    

        x1_pos = F1(time)
        x2_pos = F2(time)

        points1.append(x1_pos)
        points1 = points1[-10:]
        s1_main.set_offsets([[x1_pos, -.2 ]])
        s1_points.set_offsets(np.vstack((points1, -.2*np.ones(len(points1)))).T)

        points2.append(x2_pos)
        points2 = points2[-10:]
        s2_main.set_offsets([[x2_pos, .2 ]])
        s2_points.set_offsets(np.vstack((points2, .2*np.ones(len(points2)))).T)
        
        
        text.set_text("$t=%6.2f secs$"%time)
    plt.close(fig)
    anim = animation.FuncAnimation(fig, update, np.arange(t_max)*1./10) #len(dtimes.keys()))
    anim.save(fname, writer="ffmpeg", fps=20, bitrate=1800)

## Create an animation with different $\omega_0$

In [4]:
params1 = { "omega0": 1, "xi": 0.05, "init_pos": 1, "init_vel": 0 }
params2 = { "omega0": 2, "xi": 0.05, "init_pos": 1, "init_vel": 0 }

generate_animation("anims/anim-damped-omega.mp4", params1, params2, t_max=200, x_range=(-2,2))

frame number: 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200


## Create an animation with different $\xi$

In [5]:
params1 = { "omega0": 1, "xi": 0.05, "init_pos": 1, "init_vel": 0 }
params2 = { "omega0": 1, "xi": 0.001, "init_pos": 1, "init_vel": 0 }

generate_animation("anims/anim-damped-chi.mp4", params1, params2, t_max=200, x_range=(-2,2))

frame number: 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200
